# Hello, World
The point of this notebook is to explore Udi's design choices, constraints, etc.  

For example, by looking at his data processing code in `src/data/`, it looks as if Udi basically
begins his work at 2 previously processed files:
* `../data/raw/BISH FETAL GA ASSESSMENT.csv`
* `../data/interim/mbh001_Data-Wide-QC-full-data_20190924_V4_feateng.p`

I spent a lot of time going over this code and the data it derives from,  
however, by reading through his project's README, it turns out that he has a step before 
this...which is the step that connect Udi's pipeline to Carlos' pipeline.

In Udi's repo, he has this listed as the first step in his pipeline.  

> ### Script
> Rscript src/features/build_features.py
> 
> ### Input:
>
> * **Features metadata**: data/processed/mbh001_Table-feature-timepoints_2090613_V8.xlsx 
>   - which features to keep for F3 > baseline model
> * **Full sensor data**: data/raw/mbh001_Data-Wide-QC-full-data_20190614_V1.csv 
>   - The wide QC'ed data from mbh001_cu project
> * **High quality Toitu data**: data/raw/Fetal_Toitu_Physio_Features_Early_Signal_093019.csv
> * **High quality smoking and alcohol data**: data/raw/File_Early_Signal_alcohol_nicolo_20190924.xlsx
>
> ### Output
> data/interim/mbh001_Data-Wide-QC-full-data_20190924_V4_feateng.p (and csv)
>
> ### Notes
>
> * Keeping only features collected up to F3 based on information in the features metadata file
> * Incorporating high quality Toi features data file
> * Incorporating high quality alcohol and smoking features data file
> * Removing redundant weight and height features
> * Merging anxiety (adstai db) and depression (adedin db) features collected at F1 , F2 and F3 to 
>   eliminate missing values. A new feature is created to indicated from which cycle the data was recorded




Importantly, viewing the pipeline as the sequence of transformations from the raw data, 
Udi's first step can be better described as the pipeline's 5th step since 
Udi picks up from Carlos' "Step 4" (look at Carlos' README file):

> ## 4. Create wide version of the QC'd dataset
>
> ### Script
> python src/data/make_qc_data_long_to_wide.py
>
> ### Input
> ./data/processed/full_data_qc/mbh001_Data-QC-full-data_20190110_V1.feather
>
> ### Output
> ./data/processed/full_data_qc/mbh001_Data-Wide-QC-full-data_20190614_V1.csv

Udi does not directly use any of the later-stage processing from Carlos' iteration of the project...though
he does use some of the concepts/ideas from Step 5, where Carlos utilizes a table he and/or Udi
made that depicts which features belong to what stage/phase/cycle of
the pregnancy.  In Step 5, Carlos uses the same inputs as Udi (the Data-Wide-QC-full-data table
and the time-points table)...so I'm guessing Udi re-uses a lot of Carlos' code at this point,
but has customized to meet his needs and respect whatever lessons were learned in his time
spent on the project.

Ok, so recap:  Udi's first step is a re-hash/customization of Carlos' 5th step.  Udi takes
the Data-Wide-QC table from Carlos' first step, then adds 
in high quality TOI features, as well as high quality smoking/alcohol features (I'm
guessing an remnant TOI/smoking/alcohol features from Carlos' table are dropped in this stage).

In summary, the files that Udi uses are:

* data/processed/`mbh002_f3_toi_baseline_a.csv` AND data/processed/`mbh002_f3_toi_mon_baseline_a.csv`, 
  **which derive from**:
    - data/raw/`BISH FETAL GA ASSESSMENT.csv`
    - data/interim/`mbh001_Data-Wide-QC-full-data_20190924_V4_feateng.p`, **which derives from**:
      * data/processed/`mbh001_Table-feature-timepoints_2090613_V8.xlsx`
      * data/raw/`Fetal_Toitu_Physio_Features_Early_Signal_093019.csv`
      * data/raw/`File_Early_Signal_alcohol_nicolo_20190924.xlsx`
      * data/raw/`mbh001_Data-Wide-QC-full-data_20190614_V1.csv`, **which derives from**:
          - scripts in Carlos' 4th step (explored more below)


# Carlos' Pipeline


## P1 (Carlos): Join All (Raw) Clinical and Demographic Data into One Demo/Clinical Table
Carlos' pipeline starts with ~33 xlsx and csv files (referred to as databases) that are stored in
his repo's data/raw/demographics directory (which is a slight misnomer given that many of the data
files store clinical information, such as mental health scores, etc).  In his README file,
Carlos has the following table depicting what the starting files are, along with the database name, and
sometimes a comment.

|file                                       |database                                      |Comment |
|:------------------------------------------|:---------------------------------------------|:-------|
|adabr.xlsx                                 |ADABR                                         |        |
|adafppappa.xlsx                            |ADAFPPAPPA                                    |        |
|IMP_adalc_patid_BISH_FINAL.csv             |ADALC_PATID                                   | Imputed by CU|
|adamiel.xlsx                               |ADAMIEL                                       |        |
|adamielbaby.xlsx                           |ADAMIELBABY                                   |        |
|adcdrisc.xlsx                              |ADCDRISC                                      |        |
|adce.xlsx                                  |ADCE                                          |        |
|adcebaby.xlsx                              |ADCEBABY                                      |        |
|adcompl.xlsx                               |ADCOMPL                                       |        |
|addrg_patid.xlsx                           |ADDRG_PATID                                   |        |
|addysmo.xlsx                               |ADDYSMO                                       |        |
|adedin_merged_edit_bish.xlsx               |ADEDIN                                        |        |
|adelig.xlsx                                |ADELIG                                        |        |
|adexp_patid.xlsx                           |ADEXP_PATID                                   |        |
|adfetalgrowth_BISHCORRECTED_01.24.19.xlsx  |ADFETALGROWTH                                 |QC'd by CU, features computed in `src/data/make_efw_feature.R` |
|adinfntspecimenbaby.xlsx                   |ADINFNTSPECIMENBABY                           |        |
|adinfntvs_long.xlsx                        |ADINFNTVS_LONG                                |        |
|adinfntvs.xlsx                             |ADINFNTVS                                     |        |
|admatspecimen.xlsx                         |ADMATSPECIMEN                                 |        |
|admecon_smkdrg.xlsx                        |ADMECON_SMKDRG                                |        |
|admeconium.xlsx                            |ADMECONIUM                                    |        |
|admh.xlsx                                  |ADMH                                          |        |
|admull.xlsx                                |ADMULL                                        |        |
|adoae.xlsx                                 |ADOAE                                         |        |
|adplacenta.xlsx                            |ADPLACENTA                                    |        |
|adptsd.xlsx                                |ADPTSD                                        |        |
|adreenroll.xlsx                            |ADREENROLL                                    |        |
|adsc.xlsx                                  |ADSC                                          |        |
|adscmat.xlsx                               |ADSCMAT                                       |        |
|adsmk_patid.xlsx                           |ADSMK_PATID                                   |        |
|adstai.xlsx                                |ADSTAI                                        |        |
|advs_long.xlsx                             |ADVS_LONG                                     |        |
|advs.xlsx                                  |ADVS                                          |        |


For this step of the pipeline, Carlos runs `Rscript src/data/make_demographics-full_data.R`, which ingests all 
of the above tables and spits out `data/interim/demographics/mbh001_Data-full-demographics-long_20181221_V2.csv`. This table does not yet have any sensor data.  It is referred to as long b/c it does not yet respect
the typical rules of a "wide" format (e.g., see: https://en.wikipedia.org/wiki/Wide_and_narrow_data).  


## P2 (Carlos):  Join All (Raw) Sensor Data One Sensor Table
Similarly for sensors, Carlos and Udi received a handful of files that they had to harmonize and
join together.  This table is included in the README to depict the storage locations of the various
sensor data sources:

|path                                                              |Database                |
|:-----------------------------------------------------------------|:-----------------------|
|data/raw/fetal_hr/BISH_RMSSD_MONICA_052218_EARLY_SIGNAL2.csv               |MON HR and RMMSD        |
|data/raw/maternal_hr/Maternal HR XFER 052318.xlsx                          |MON HR Signal Features  |      
|data/raw/movement/maternal_MVMT_monica_EHG_EHG_XFER_240s_Epochs_051818.csv |MON MVT Signal Features |
|data/raw/movement/fetal_MVMT_toitu_EDIT_240s_XFER_Signal_051818.csv        |TOI HR and MVT          |

This step of the pipeline is, in a sense, paralell to the demo/clinical step since two separate
collated tables are being created, which are then also combined down the line.  

For this step in the pipeline, Carlos runs `Rscript src/data/make_sensor-full_data.R`, which
ingests all the sensor tables listed above and spits out `data/interim/sensors/mbh001_Data-sensors-long_20181019_V1.csv`.  Here, Carlos also notes that here's a mistake in the variable names in `data/raw/sensors/movement/maternal_MVMT_monica_EHG_EHG_XFER_240s_Epochs_051818.csv`: the features labelled as HR correspond to MMVT.

## P3 (Carlos):  QC All Data
In this step, the results of the previous two steps converge along with a QC table
that Udi/Carlos put together, which lists acceptable ranges for each var:
* **Min/Max QC Table**: data/processed/qc_values⁩/mbh001_Table-Summary-Numeric_Features_20181221_V2.xlsx
* **Joined Sensor Table**: data/interim/demographics/mbh001_Data-full-demographics-long_20181221_V2.csv
* **Joined Demo/Clinical Table**: data/interim/sensors/mbh001_Data-sensors-long_20181019_V1.csv

These tables are ingested by running `Rscript src/data/make_qc_data.R`, which then spits
out a "Long QC" table: `data/processed/full_data_qc/mbh001_Data-QC-full-data_20190110_V1.csv`.  Carlos
notes that artifcats are denoted by `-999999`, however Udi told me before he left that he
had to modify a downstream script to search for both `-999999` (six 9's) and `-99999` (five 9's).  

##  P4 (Carlos):  Transform the Long QC Table to a Wide QC Table
This step is as straightforward as the title suggests: Carlos uses `python src/data/make_qc_data_long_to_wide.py`
to ingest the long qc table (`data/processed/full_data_qc/mbh001_Data-QC-full-data_20190110_V1.feather`) and
spit out the wide qc tab (`data/processed/full_data_qc/mbh001_Data-Wide-QC-full-data_20190614_V1.csv`).  

**NOTE**: For anyone reading this wondering what the various `.p` and `.feather` 
extensions are, just know they simply 
correspond to more efficient encodings of the data.  In the above outline of the pipeline, you can still 
think of them as if they were CSV files if you want -- the gist is that tables are being built and stored in
one stage of the pipeline, then loaded into another scripting environment in an subsequent stage.  If you
actually want to look at the data and are unsure of how to load `.p` or `.feather` files, then no worries:
Udi and Carlos often have a corresponding CSV file saved to.

**Subsequent Pipeline Stages in Carlos' Original Pipeline**:
Carlos' pipeline goes onto do quite a few other things, however Udi does not use any
of those things directly.  Instead, he forks the project here and starts his own
repo `mbh001_cu_2`.

## P5 (Udi):  Touch Up Carlos' "Wide QC Full Data" Table

> ### Script
> Rscript src/features/build_features.py
> 
> ### Input:
>
> * **Features metadata**: data/processed/mbh001_Table-feature-timepoints_2090613_V8.xlsx 
>   - which features to keep for F3 > baseline model
> * **Full sensor data**: data/raw/mbh001_Data-Wide-QC-full-data_20190614_V1.csv 
>   - The wide QC'ed data from mbh001_cu project
> * **High quality Toitu data**: data/raw/Fetal_Toitu_Physio_Features_Early_Signal_093019.csv
> * **High quality smoking and alcohol data**: data/raw/File_Early_Signal_alcohol_nicolo_20190924.xlsx
>
> ### Output
> data/interim/mbh001_Data-Wide-QC-full-data_20190924_V4_feateng.p (and csv)
>
> ### Notes
>
> * Keeping only features collected up to F3 based on information in the features metadata file
> * Incorporating high quality Toi features data file
> * Incorporating high quality alcohol and smoking features data file
> * Removing redundant weight and height features
> * Merging anxiety (adstai db) and depression (adedin db) features collected at F1 , F2 and F3 to 
>   eliminate missing values. A new feature is created to indicated from which cycle the data was recorded

## P6 (Udi)

Note that Udi has some typos in his README, some of which I fixed w/o explicitly noting.  But here
is one:  here, he lists the script in use as `Rscript src/features/build_data.py`, when in reality
it is `python src/data/build_f3_data.py`.

> ### Script
> `python src/data/build_f3_data.py`
>
> ### Input 
> * Interim data: `data/interim/mbh001_Data-Wide-QC-full-data_20190924_V4_feateng.p`
> * F3 recording GA : `data/raw/BISH FETAL GA ASSESSMENT.csv`
>
> ### Output 
> * Baseline TOI and Monica file (Copmlete TOI intersected with Mon missing in less than 55% of the cases: 
>   `data/processed/mbh001_f3_toi_mon_baseline_timestamp.p` (and csv)
> * Baseline TOI (Complete TOI): `data/processed/mbh001_f3_toi_baseline_timestamp.p` (and csv)
> * Feature profile Toi  (Summary statistics and NAs): 
>   `data/processed/mbh001_f3_toi_baseline_timestamp_feat_profile.csv` 
> * Feature profile Toi and Mon (Summary statistics and NAs): 
>   `data/processed/mbh001_f3_toi_mon_baseline_timestamp_feat_profile.csv` 

Much of my analysis below looks at this code and the data being used from input files.

# Udi's Data Files

In [1]:
!ls ../data/raw

BISH FETAL GA ASSESSMENT.csv
Fetal_Toitu_Physio_Features_Early_Signal_093019.csv
File_Early_Signal_alcohol_nicolo_20190924.xlsx
f3_df.csv
jdn_edits_092319_mbh002_feat-profile-df_20190913_v01.csv
mbh001_Data-Train-Test-GA-20190501_V1.p
mbh001_Data-Wide-QC-full-data_20190614_V1.csv
mbh001_data-train-and-test-final-report_v01.p


In [2]:
!ls ../data/interim/

mbh001_Data-Wide-QC-full-data_20190614_V1_feateng.csv
mbh001_Data-Wide-QC-full-data_20190919_V2_feateng.csv
mbh001_Data-Wide-QC-full-data_20190924_V3_feateng.csv
mbh001_Data-Wide-QC-full-data_20190924_V4_feateng
mbh001_Data-Wide-QC-full-data_20190924_V4_feateng.csv
mbh001_Data-Wide-QC-full-data_20190924_V4_feateng.p


In [2]:
!ls ../data/processed/

feat_profile.csv
feat_profile1.csv
feat_profile_artifacts_as_na_20190923_V01.csv
feat_profile_df.csv
feat_profile_df_20191909_v01.csv
feat_profile_df_20192409_v01.csv
feat_profile_df_artifacts_as_na.csv
feat_profile_df_for_slides.csv
feat_profile_df_for_slides.xlsx
feature_profiling.pdf
mbh001_Data-QC-full-data_20190110_V1.feather
mbh001_Data-Train-Test-GA-20190501_V1.p
mbh001_Table-feature-timepoints_2090613_V8 copy.xlsx
mbh001_Table-feature-timepoints_2090613_V8.xlsx
mbh001_data-train-and-test-final-report_v01.p
mbh001_f3_toi_baseline_a.csv
mbh001_f3_toi_baseline_a.p
mbh001_f3_toi_baseline_a_feat_profile.csv
mbh001_f3_toi_mon_baseline_a.csv
mbh001_f3_toi_mon_baseline_a.p
mbh001_f3_toi_mon_baseline_a_feat_profile.csv
mbh002_Table-feature-timepoints_2090613_V5.xlsx
mbh002_Table-feature-timepoints_2090613_V6.xlsx
mbh002_Table-feature-timepoints_2090613_V7.xlsx
mbh002_Table-feature-timepoints_2090613_V8.xlsx
mbh002_f3_baseline.csv
mbh002_f3_baseline_20190924202444.csv
mbh002_f3_baseline_

# Udi's Data Processing Scripts

In [2]:
!tree ../src/

../src/
├── __init__.py
├── data
│   ├── __init__.py
│   ├── build_f3_data.py
│   ├── feat_profile.py
│   └── make_dataset.py
├── features
│   ├── __init__.py
│   └── build_features.py
├── models
│   ├── __init__.py
│   ├── predict_model.py
│   └── train_model.py
└── visualization
    ├── __init__.py
    └── visualize.py

4 directories, 12 files


In [3]:
!ls ../src/data

__init__.py      build_f3_data.py feat_profile.py  make_dataset.py


# Investigating Udi's First Step (P5)

In [253]:
!cat ../src/features/build_features.py

import pandas as pd
import numpy as np
import pickle


#------------------------------------------------------------------------------
# Extracting F3 Features
#------------------------------------------------------------------------------
# Load in Feature Time Points File
feat_timepoint = pd.read_excel(
        "data/processed/mbh001_Table-feature-timepoints_2090613_V8.xlsx")
# Only keep F3 Baseline Records
#  -- also: only keep necessary cols (e.g., for naming convention)
feat_timepoint_f3 = feat_timepoint[
        feat_timepoint['KEEP_F3_BASELINE']==1].\
                loc[:,['DB','feature','cycleid','feat_type']]
# Create features names following DB.FTR.CID convention
f3_features = feat_timepoint_f3['DB'] + '.' + feat_timepoint_f3['feature'] +\
        '.' + feat_timepoint_f3['cycleid'].astype('str')
# Create DataFrame that matches feature names to data type
f3_features_df = pd.DataFrame({
    'feat_name': f3_features,
    'feat_type': feat_timepoint_f3['feat_type']}).set_index('

In [14]:
import pandas as pd
import numpy as np
from datetime import datetime
import pickle

In [16]:
toihq = pd.read_csv("../data/raw/Fetal_Toitu_Physio_Features_Early_Signal_093019.csv")
toihq.columns = toihq.columns.map(str.lower)
toihq.set_index('patid',inplace=True)

### What is the TOI table shaped like?
We find here that there are 14,157 rows, however this is wrong.  For some reason, over 14,000 rows are imported from the TOI CSV file.  However, when you inspect more closely, most of the rows (over 7,000) are 100% empty.  

In [18]:
toihq.shape

(14157, 47)

### How many unique patient IDs?
Assuming you do not know that over 7k rows are empty, if you compare the number of unique 
patients IDs to the number of rows, it would appear that patients typically have more than
one record in the TOI data.

In [29]:
toihq_nrows = len(toihq.index)
toihq_nrows_uniq = len(toihq.index.unique())
print('Num Rows:\t\t', toihq_nrows)
print('Num Distinct Patients:\t', toihq_nrows_uniq)

Num Rows:		 14157
Num Distinct Patients:	 6492


### How many unique patient ID row counts exist?
Still assuming there are 14k rows, but only 6.5k patients, you might be interested
in knowing 
* how many patients have 1 row of data?
* how many have 2 rows of data?
* how many have 3 rows of data?
* ...and so on.

Here we find that a patient either has 1 record or 2.  That's it.

Ok... So most patients MUST have 2 rows.  Again, assuming nothing was wrong with the 
loaded data table, we have 6.5k patients and 14k records...

But, wait!  That doesn't make sense, right?  Basic math: `2 * 6.5k = 13k`.  In other words,
even if all the patients had 2 rows, this would account for all the rows in the file.

In [147]:
patid_count = toihq.reset_index().groupby('patid')['patid'].agg('count').to_frame('count')
patid_count.reset_index().groupby('count').agg('count')

,patid
count,
1,6489
2,2


In [151]:
patid_count.head(3)

,count
patid,
S002-BISH-00001,1
S002-BISH-00002,1
S002-BISH-00003,1


Something is fishy... Let's try counting duplicated patient IDs again... 

In [154]:
len([item for item in patid_count.iterrows() if item[1]['count'] < 1])

0

In [155]:
len([item for item in patid_count.iterrows() if item[1]['count'] == 1])

6489

In [156]:
len([item for item in patid_count.iterrows() if item[1]['count'] == 2])

2

In [157]:
len([item for item in patid_count.iterrows() if item[1]['count'] > 2])

0

#### Further Investigation
At this point, I should have just opened Excel, which makes it REAL obvious that 
there are only around 6.5k rows of real data...the other rows being fully empty.  

But I didn't!

I went back to the toihq dataframe and got an important ERROR message that helped
me crack the case. 

Desperate, I figured I'd check whether some IDs are upper case and other lower case... Not
that this should have produced any fruitful results (they would have just been treated
as distinct IDs), but as luck would have it, it did anyway.

Turns out that 7,664 PATIENT IDs are NaNs... This is what finally led me to opening
up Excel.  Sometimes it's the best tool!

In [178]:
[item for item in toihq.index][:10]

['S002-BISH-00001',
 'S002-BISH-00002',
 'S002-BISH-00010',
 'S002-BISH-00012',
 'S002-BISH-00015',
 'S002-BISH-00016',
 'S002-BISH-00017',
 'S002-BISH-00020',
 'S002-BISH-00022',
 'S002-BISH-00026']

In [179]:
[item.lower() for item in toihq.index][:10]

AttributeError: 'float' object has no attribute 'lower'

In [180]:
len([item for item in toihq.index if type(item)==float])

7664

In [182]:
len([item for item in toihq.index if type(item)!=str])

7664

In [187]:
{item for item in toihq.index if type(item)!=str}

{nan}

In [189]:
len(toihq)-7664

6493

### REMOVE ALL NaN INDICES FORM TOI HQ 

In [201]:
toihq = toihq.reset_index().dropna(subset=['patid']).set_index('patid')

In [202]:
toihq.index

Index(['S002-BISH-00001', 'S002-BISH-00002', 'S002-BISH-00010',
       'S002-BISH-00012', 'S002-BISH-00015', 'S002-BISH-00016',
       'S002-BISH-00017', 'S002-BISH-00020', 'S002-BISH-00022',
       'S002-BISH-00026',
       ...
       'S002-BISH-06375', 'S002-BISH-06610', 'S002-BISH-06632',
       'S002-BISH-06647', 'S002-BISH-06715', 'S002-BISH-06780',
       'S002-BISH-06855', 'S002-BISH-06883', 'S002-BISH-06921',
       'S002-BISH-06940'],
      dtype='object', name='patid', length=6493)

### LOOK OVER EVERYTHING AGAIN ON FIXED TOI TABLE

In [204]:
toihq.shape

(6493, 47)

In [205]:
toihq_nrows = len(toihq.index)
toihq_nrows_uniq = len(toihq.index.unique())
print('Num Rows:\t\t', toihq_nrows)
print('Num Distinct Patients:\t', toihq_nrows_uniq)

Num Rows:		 6493
Num Distinct Patients:	 6491


In [206]:
patid_count = toihq.reset_index().groupby('patid')['patid'].agg('count').to_frame('count')
patid_count.reset_index().groupby('count').agg('count')

,patid
count,
1,6489
2,2


In [207]:
patid_count.query('count==2')

,count
patid,
S002-BISH-04030,2
S002-BISH-04033,2


### Duplicate Patient #1

---------------------------

In [228]:
dup1 = toihq.loc['S002-BISH-04030']
dup1_eq_index = dup1.iloc[0] == dup1.iloc[1]
dup1_ne_index = dup1.iloc[0] != dup1.iloc[1]

In [229]:
dup1.loc[:,dup1_eq_index.tolist()]

,f2_hrmean1,f2_hrmed1,f2_hrstd1,f2_hrsegs1,f2_totsegs1,f2_totsegs4,f2_ccsegs4,f2_mvt_pwr_numbersegments.1,f2_mvt_pwr_numberusedpwrcoh,f31f_hrstd1,...,f32f_totsegs1,f3_medcc4,f3_medlag4,f3_totsegs4,f3_ccsegs4,f3_mvt_pwr_mvt21,f3_mvt_pwr_mvt31,f3_mvt_pwr_mvt41,f3_mvt_pwr_numbersegments.1,f3_mvt_pwr_numberusedpwrcoh
patid,,,,,,,,,,,,,,,,,,,,,
S002-BISH-04030,142.0795,141.754,3.752135,4.0,10.0,2.0,0.0,2.0,0.0,2.53636,...,50.0,0.796676,-7.7,12.0,5.0,0.249382,0.138689,0.360455,12.0,7.0
S002-BISH-04030,142.0795,141.754,3.752135,4.0,10.0,2.0,0.0,2.0,0.0,2.53636,...,50.0,0.796676,-7.7,12.0,5.0,0.249382,0.138689,0.360455,12.0,7.0


In [230]:
dup1.loc[:,dup1_ne_index.tolist()]

,f1_hrmean1,f1_hrmed1,f1_hrstd1,f1_hrsegs1,f1_totsegs1,f1_medcc4,f1_medlag4,f1_totsegs4,f1_ccsegs4,f1_mvt_pwr_mvt21,...,f1_mvt_pwr_mvt41,f1_mvt_pwr_numbersegments.1,f1_mvt_pwr_numberusedpwrcoh,f2_medcc4,f2_medlag4,f2_mvt_pwr_mvt21,f2_mvt_pwr_mvt31,f2_mvt_pwr_mvt41,f31f_hrmean1,f31f_hrmed1
patid,,,,,,,,,,,,,,,,,,,,,
S002-BISH-04030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,134.4960,134.2735
S002-BISH-04030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,134.9185,134.6745


In [231]:
dup1.loc[:,dup1_ne_index.tolist()].dropna(axis=1)

,f31f_hrmean1,f31f_hrmed1
patid,,
S002-BISH-04030,134.4960,134.2735
S002-BISH-04030,134.9185,134.6745


### Duplicate Patient #2

------------------------------------

In [232]:
dup2 = toihq.loc['S002-BISH-04033']
dup2_eq_index = dup2.iloc[0] == dup2.iloc[1]
dup2_ne_index = dup2.iloc[0] != dup2.iloc[1]

In [233]:
dup2.loc[:,dup2_eq_index.tolist()]

,f2_hrmean1,f2_hrmed1,f2_hrstd1,f2_hrsegs1,f2_totsegs1,f2_medcc4,f2_medlag4,f2_totsegs4,f2_ccsegs4,f2_mvt_pwr_mvt21,...,f32f_totsegs1,f3_medcc4,f3_medlag4,f3_totsegs4,f3_ccsegs4,f3_mvt_pwr_mvt21,f3_mvt_pwr_mvt31,f3_mvt_pwr_mvt41,f3_mvt_pwr_numbersegments.1,f3_mvt_pwr_numberusedpwrcoh
patid,,,,,,,,,,,,,,,,,,,,,
S002-BISH-04033,150.453,149.597,3.80242,33.0,40.0,0.695448,-9.15,11.0,2.0,0.240248,...,50.0,0.516059,-10.15,12.0,3.0,0.087863,0.129704,0.476141,12.0,8.0
S002-BISH-04033,150.453,149.597,3.80242,33.0,40.0,0.695448,-9.15,11.0,2.0,0.240248,...,50.0,0.516059,-10.15,12.0,3.0,0.087863,0.129704,0.476141,12.0,8.0


In [234]:
dup2.loc[:,dup2_ne_index.tolist()]

,f1_hrmean1,f1_hrmed1,f1_hrstd1,f1_hrsegs1,f1_totsegs1,f1_medcc4,f1_medlag4,f1_totsegs4,f1_ccsegs4,f1_mvt_pwr_mvt21,...,f1_mvt_pwr_numberusedpwrcoh,f31f_hrmean1,f31f_hrmed1,f31f_hrstd1,f31f_hrsegs1,f31f_totsegs1,f32f_hrmean1,f32f_hrmed1,f32f_hrstd1,f32f_hrsegs1
patid,,,,,,,,,,,,,,,,,,,,,
S002-BISH-04033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,151.5215,150.8220,3.88475,24.0
S002-BISH-04033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,156.7475,157.0055,3.96170,41.0


In [235]:
dup2.loc[:,dup2_ne_index.tolist()].dropna(axis=1)

,f32f_hrmean1,f32f_hrmed1,f32f_hrstd1,f32f_hrsegs1
patid,,,,
S002-BISH-04033,151.5215,150.8220,3.88475,24.0
S002-BISH-04033,156.7475,157.0055,3.96170,41.0


--------------------------------------
**My 2 cents**:

Since there are only 2 patients that are duplicates, and since one of them has
3 features that are very different... Let's just drop them.  Without a doubt
drop the troublesome duplicate:  how to choose which is the right data record?

------------------------------------------

### What data is in the TOI table?
Remember: in the conversations between Udi and David, David said that the only 
features he considered to be high quality were the band power features -- and that's
only if Udi decided to normalize them by the total power...  

In the columns below, none of them have the band power names that David mentioned... Closest
thing there is are vars like `f1_mvt_pwr_mvt21`.

In [203]:
toihq.columns

Index(['f1_hrmean1', 'f1_hrmed1', 'f1_hrstd1', 'f1_hrsegs1', 'f1_totsegs1',
       'f1_medcc4', 'f1_medlag4', 'f1_totsegs4', 'f1_ccsegs4',
       'f1_mvt_pwr_mvt21', 'f1_mvt_pwr_mvt31', 'f1_mvt_pwr_mvt41',
       'f1_mvt_pwr_numbersegments.1', 'f1_mvt_pwr_numberusedpwrcoh',
       'f2_hrmean1', 'f2_hrmed1', 'f2_hrstd1', 'f2_hrsegs1', 'f2_totsegs1',
       'f2_medcc4', 'f2_medlag4', 'f2_totsegs4', 'f2_ccsegs4',
       'f2_mvt_pwr_mvt21', 'f2_mvt_pwr_mvt31', 'f2_mvt_pwr_mvt41',
       'f2_mvt_pwr_numbersegments.1', 'f2_mvt_pwr_numberusedpwrcoh',
       'f31f_hrmean1', 'f31f_hrmed1', 'f31f_hrstd1', 'f31f_hrsegs1',
       'f31f_totsegs1', 'f32f_hrmean1', 'f32f_hrmed1', 'f32f_hrstd1',
       'f32f_hrsegs1', 'f32f_totsegs1', 'f3_medcc4', 'f3_medlag4',
       'f3_totsegs4', 'f3_ccsegs4', 'f3_mvt_pwr_mvt21', 'f3_mvt_pwr_mvt31',
       'f3_mvt_pwr_mvt41', 'f3_mvt_pwr_numbersegments.1',
       'f3_mvt_pwr_numberusedpwrcoh'],
      dtype='object')

In [20]:
toihq.head(3)

,f1_hrmean1,f1_hrmed1,f1_hrstd1,f1_hrsegs1,f1_totsegs1,f1_medcc4,f1_medlag4,f1_totsegs4,f1_ccsegs4,f1_mvt_pwr_mvt21,...,f32f_totsegs1,f3_medcc4,f3_medlag4,f3_totsegs4,f3_ccsegs4,f3_mvt_pwr_mvt21,f3_mvt_pwr_mvt31,f3_mvt_pwr_mvt41,f3_mvt_pwr_numbersegments.1,f3_mvt_pwr_numberusedpwrcoh
patid,,,,,,,,,,,,,,,,,,,,,
S002-BISH-00001,147.1780,148.1590,1.95969,16.0,34.0,0.471563,-14.5,8.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
S002-BISH-00002,148.9275,149.0025,3.25205,41.0,51.0,0.698840,-7.9,13.0,9.0,0.224079,...,50.0,0.668929,-4.55,12.0,5.0,0.147417,0.096355,0.498982,12.0,8.0
S002-BISH-00010,142.0905,142.3115,2.23203,52.0,56.0,NaN,NaN,14.0,0.0,0.100899,...,NaN,NaN,NaN,NaN,NaN,0.073736,0.112650,0.512278,12.0,3.0


## Let's Go Through the Code

In [242]:
#------------------------------------------------------------------------------
# Extracting F3 Features
#------------------------------------------------------------------------------
# Load in Feature Time Points File
feat_timepoint = pd.read_excel(
        "../data/processed/mbh001_Table-feature-timepoints_2090613_V8.xlsx")
# Only keep F3 Baseline Records
#  -- also: only keep necessary cols (e.g., for naming convention)
feat_timepoint_f3 = feat_timepoint[
        feat_timepoint['KEEP_F3_BASELINE']==1].\
                loc[:,['DB','feature','cycleid','feat_type']]
# Create features names following DB.FTR.CID convention
f3_features = feat_timepoint_f3['DB'] + '.' + feat_timepoint_f3['feature'] +\
        '.' + feat_timepoint_f3['cycleid'].astype('str')
# Create DataFrame that matches feature names to data type
f3_features_df = pd.DataFrame({
    'feat_name': f3_features,
    'feat_type': feat_timepoint_f3['feat_type']}).set_index('feat_name')
# Ensure all feature names are lower case
f3_features_df.index = (f3_features_df.index).map(str.lower)
# Index all numeric features
num_f3_feat = f3_features_df[f3_features_df['feat_type'] == 'numeric'].index
# Index all categorical features
cat_f3_feat = f3_features_df[f3_features_df['feat_type'] == 'categorical'].index

In [262]:
#------------------------------------------------------------------------------
# Loading Wide-QC'd Data
#------------------------------------------------------------------------------
# Load in QC'd, Wide, Full Dta Table (output of Carlos' step 4)
df1 = pd.read_csv("../data/raw/mbh001_Data-Wide-QC-full-data_20190614_V1.csv",
                dtype = {754: str, 757: str, 765: str, 855: str, 1225: str,
                         1226: str, 1229: str, 1475: str, 1485: str}).\
        set_index('patid', drop=False)
# Keep only the F3 features (identified above using time points table)
df1 = df1[f3_features]
# Ensure that column names are all lower case
df1.columns = map(str.lower, df1.columns)
# Ensure that all numeric features are of type float64
df1[num_f3_feat] = df1[num_f3_feat].astype('float64')
# Ensure that all categorical features are of type object
df1[cat_f3_feat] = df1[cat_f3_feat].astype('object')
# Keep Only Numeric and Categorical (Ignore Date Vars)
df1 = df1[num_f3_feat.union(cat_f3_feat)]

### Breakdown Prior to Merging TOI Data

In [263]:
df1.shape

(6975, 563)

In [264]:
df1.columns

Index(['adafppappa.afpga.10', 'adafppappa.afpga.15', 'adafppappa.afpga.pre10',
       'adafppappa.afpmom.10', 'adafppappa.afpmom.15',
       'adafppappa.afpmom.pre10', 'adafppappa.pappaga.10',
       'adcdrisc.cdrisc_nmiss.15', 'adcdrisc.cdrisc_nmiss.20',
       'adcdrisc.cdrisc_raw.15',
       ...
       'mon.mmvt_mean.20', 'mon.mmvt_median.10', 'mon.mmvt_median.15',
       'mon.mmvt_median.20', 'mon.mmvt_min.10', 'mon.mmvt_min.15',
       'mon.mmvt_min.20', 'mon.mmvt_stddev.10', 'mon.mmvt_stddev.15',
       'mon.mmvt_stddev.20'],
      dtype='object', length=563)

In [265]:
df1.filter(regex='.+toi.+').columns

Index(['adscmat.toiletwater2.any'], dtype='object')

In [266]:
print('Number of Rows:', df1.shape[0])
print('Number of Unique Patient IDs', len(df1.index.unique()))

Number of Rows: 6975
Number of Unique Patient IDs 6925


### Continue w/ Code

In [267]:
toihq = pd.read_csv("../data/raw/Fetal_Toitu_Physio_Features_Early_Signal_093019.csv")
toihq.columns = toihq.columns.map(str.lower)
toihq.set_index('patid', inplace=True)

def toi_feat_names(
    x,
):
    '''
    Converts feature names:
        f1_mvmt11sd to TOI.mvmt11sd.10
        f2_mvmt11sd to TOI.mvmt11sd.15
        f3_mvmt11sd to TOI.mvmt11sd.20
    '''
    cycle = x[0:2]
    if cycle == 'f1':
        return('toi.' + x + '.10')
    elif cycle == 'f2':
        return('toi.' + x + '.15')
    elif cycle == 'f3':
        return('toi.' + x + '.20')
    else:
        return(x)

toihq.columns = toihq.columns.map(lambda x: toi_feat_names(x))
toihq = toihq.astype('float64')
df2 = df1.merge(toihq, left_index=True, right_index=True, how='left')

In [275]:
print('Shape before merge:', df1.shape)
print('Shape after merge:', df2.shape)

Shape before merge: (6975, 563)
Shape after merge: (6977, 610)


In [278]:
print('Num of TOI columns:',toihq.shape[1])
print('Num of Demo/Clinical Columns:',df1.shape[1])
print('Demo/Clinical Cols + TOI Cols:', df1.shape[1] + toihq.shape[1])

Num of TOI columns: 47
Num of Demo/Clinical Columns: 563
Demo/Clinical Cols + TOI Cols: 610


In [273]:
print('Number of Rows in DemoClinical Data:', df1.shape[0])
print('Number of Rows in DemoClinical/TOI Merged Data:', df2.shape[0])
print()
print('Number of Unique Patient IDs in DemoClinical Data', len(df1.index.unique()))
print('Number of Unique Patient IDs in DemoClinical/TOI Merged Data', len(df2.index.unique()))

Number of Rows in DemoClinical Data: 6975
Number of Rows in DemoClinical/TOI Merged Data: 6977

Number of Unique Patient IDs in DemoClinical Data 6925
Number of Unique Patient IDs in DemoClinical/TOI Merged Data 6925


In [291]:
print('Number of IDs in df1, but not in df2:',len(set(df1.index).difference(df2.index)))
print('Number of IDs in df2, but not in df1:',len(set(df2.index).difference(df1.index)))

Number of IDs in df1, but not in df2: 0
Number of IDs in df2, but not in df1: 0


### Analysis
The extra two rows after the data merge likely come from the fact that TWO patient IDs 
in the TOI data set (04030 and 04033) have two records.  I identified this above,
where I also recommend to remove these two patients in our (Dani, Kevin) upcoming 
modeling efforts.  HOWEVER, note that the Demo/Clinical data set by itself has
6,975 records, but only 6,925 unique patient IDs.  This means the Demo/Clinical data
set itself has some redundant records that we should look into (BELOW).

What we find (BELOW) is that 6,875 patients have one record in the Demo/Clinical data, while 50 
patients have two records (6,875 + 50 = 6,925 = Number of Unique Patient IDs).

At least 3 of the 2-record patients in the Demo/Clinical data set have twins:
* 04355
* 04582
* 05089

In [293]:
df1_patid_count = df1.reset_index().groupby('patid')['patid'].agg('count').to_frame('count')
df1_patid_count.reset_index().groupby('count').agg('count')

,patid
count,
1,6875
2,50


In [298]:
df1_2r_patients = df1_patid_count.query('count==2')

#### # Are either of the TOI 2-record patients in the Demo/Clinical 2-record patient set?


In [304]:
[item for item in ['S002-BISH-04030','S002-BISH-04033'] if item in df1_2r_patients.index]

[]

In [301]:
df1_2r_patients.index

Index(['S002-BISH-00025', 'S002-BISH-00115', 'S002-BISH-00399',
       'S002-BISH-00721', 'S002-BISH-00744', 'S002-BISH-00980',
       'S002-BISH-01204', 'S002-BISH-01375', 'S002-BISH-01474',
       'S002-BISH-01810', 'S002-BISH-01915', 'S002-BISH-01964',
       'S002-BISH-02132', 'S002-BISH-02241', 'S002-BISH-02880',
       'S002-BISH-03139', 'S002-BISH-03152', 'S002-BISH-03347',
       'S002-BISH-03476', 'S002-BISH-03540', 'S002-BISH-03649',
       'S002-BISH-03779', 'S002-BISH-03865', 'S002-BISH-03939',
       'S002-BISH-04122', 'S002-BISH-04355', 'S002-BISH-04441',
       'S002-BISH-04515', 'S002-BISH-04582', 'S002-BISH-04811',
       'S002-BISH-05046', 'S002-BISH-05058', 'S002-BISH-05089',
       'S002-BISH-05356', 'S002-BISH-05377', 'S002-BISH-05651',
       'S002-BISH-05834', 'S002-BISH-05937', 'S002-BISH-05965',
       'S002-BISH-06129', 'S002-BISH-06153', 'S002-BISH-06210',
       'S002-BISH-06259', 'S002-BISH-06653', 'S002-BISH-06689',
       'S002-BISH-06714', 'S002-BISH-067

In [314]:
dup_list = []
for pid in df1_2r_patients.index[:10]:
    dup = df1.loc[pid]
    #dup_eq_index = dup1.iloc[0] == dup1.iloc[1]
    dup_ne_index = dup.iloc[0] != dup.iloc[1]
    print(dup.loc[:,dup_ne_index.tolist()].dropna(axis=1))
    print('\n\n-------------------------------------------------------------\n')

                adsc.infraceai4.25 adsc.infraceai7.25 adsc.infracenih4.25  \
patid                                                                       
S002-BISH-00025                  2                  4                   2   
S002-BISH-00025                  4                  7                   4   

                adsc.infracenih7.25  
patid                                
S002-BISH-00025                   4  
S002-BISH-00025                   7  


-------------------------------------------------------------

                adsc.gender2.25
patid                          
S002-BISH-00115               1
S002-BISH-00115               2


-------------------------------------------------------------

Empty DataFrame
Columns: []
Index: [S002-BISH-00399, S002-BISH-00399]


-------------------------------------------------------------

                adsc.gender2.25
patid                          
S002-BISH-00721               1
S002-BISH-00721               2


----------------

In [315]:
df1_2r_patients.index

Index(['S002-BISH-00025', 'S002-BISH-00115', 'S002-BISH-00399',
       'S002-BISH-00721', 'S002-BISH-00744', 'S002-BISH-00980',
       'S002-BISH-01204', 'S002-BISH-01375', 'S002-BISH-01474',
       'S002-BISH-01810', 'S002-BISH-01915', 'S002-BISH-01964',
       'S002-BISH-02132', 'S002-BISH-02241', 'S002-BISH-02880',
       'S002-BISH-03139', 'S002-BISH-03152', 'S002-BISH-03347',
       'S002-BISH-03476', 'S002-BISH-03540', 'S002-BISH-03649',
       'S002-BISH-03779', 'S002-BISH-03865', 'S002-BISH-03939',
       'S002-BISH-04122', 'S002-BISH-04355', 'S002-BISH-04441',
       'S002-BISH-04515', 'S002-BISH-04582', 'S002-BISH-04811',
       'S002-BISH-05046', 'S002-BISH-05058', 'S002-BISH-05089',
       'S002-BISH-05356', 'S002-BISH-05377', 'S002-BISH-05651',
       'S002-BISH-05834', 'S002-BISH-05937', 'S002-BISH-05965',
       'S002-BISH-06129', 'S002-BISH-06153', 'S002-BISH-06210',
       'S002-BISH-06259', 'S002-BISH-06653', 'S002-BISH-06689',
       'S002-BISH-06714', 'S002-BISH-067

# Investigating Udi's Second Step (P6)

In [4]:
!cat ../src/data/build_f3_data.py

import pandas as pd
import numpy as np
from datetime import datetime
import pickle

#df=pd.read_csv("data/interim/mbh001_Data-Wide-QC-full-data_20190924_V3_feateng.csv").set_index('patid',drop=True)
df=pickle.load( open( "data/interim/mbh001_Data-Wide-QC-full-data_20190924_V4_feateng.p", "rb" ) )

#Extracting f3 complete toi patids
f3_toi_patids=df.filter(regex="^toi.+20").dropna(axis=0).index # Dropping rows with missing data


#GA at F3 for each patient
collect_times=pd.read_csv("data/raw/BISH FETAL GA ASSESSMENT.csv")
collect_times.columns=collect_times.columns.str.replace(' ', '')
collect_times.rename(columns = {'GA_In_Days':'ga_in_days',"PATID":'patid'},inplace=True)
#f3_collect_times['cycle'] = f3_collect_times['FinalFileName'].dropna().apply(lambda x: x.split("-")[-1])
collect_times_f3=collect_times[collect_times.ga_in_days>=238].loc[:,["ga_in_days","patid","NumberOfFetuses"]]
collect_times_f3.rename(columns = {'ga_in_days':'ga_f3'},inplace=True)


df_f3_toi=df.loc[f3_toi_patids

# Exploring the BISH File

In [7]:
bish = pd.read_csv('../data/raw/BISH FETAL GA ASSESSMENT.csv')

### BISH Schema

In [46]:
bish.head()

,Final File Name,GA_In_Days,GA at Assessment Weeks,PATID,Box Number,Original File Name,Final File Name.1,File Type,GA at Assessment,Zip File,Overwrite,Number Of Fetuses,Box Number.1
0,S002-BISH-00001-T010-10,150,"21 Weeks, 3 Days",S002-BISH-00001,2007-04,S002-BISH-00001-2007-04.physio,S002-BISH-00001-T010-10.physio,physio,"21 Weeks, 3 Days",False,True,1,2007-04
1,S002-BISH-00001-T010-15,202,"28 Weeks, 6 Days",S002-BISH-00001,2007-04,11-22-s002-bish-00001-2007-04.physio,S002-BISH-00001-T010-15.physio,physio,"28 Weeks, 6 Days",False,True,1,2007-04
2,S002-BISH-00002-T010-10,167,"23 Weeks, 6 Days",S002-BISH-00002,2007-04,S002-BISH-00002-2007-04.physio,S002-BISH-00002-T010-10.physio,physio,"23 Weeks, 6 Days",False,True,1,2007-04
3,S002-BISH-00002-T010-20,242,"34 Weeks, 4 Days",S002-BISH-00002,2007-05,10-24-s002-bish-00002-2007-05.physio,S002-BISH-00002-T010-20.physio,physio,"34 Weeks, 4 Days",False,True,1,2007-05
4,S002-BISH-00003-T010-15,201,"28 Weeks, 5 Days",S002-BISH-00003,2007-06,S002-BISH-00003-9-17-2007-06.physio,S002-BISH-00003-T010-15.physio,physio,"28 Weeks, 5 Days",False,True,1,2007-06


### BISH's GA Tukey Summary

In [80]:
tukey = bish.GA_In_Days.describe().to_frame().iloc[3:,:]
tukey['GA_In_Weeks'] = round(tukey.GA_In_Days/7, 1)
tukey

,GA_In_Days,GA_In_Weeks
min,116.0,16.6
25%,161.0,23.0
50%,221.0,31.6
75%,242.0,34.6
max,339.0,48.4


### BISH PATID:  Rows per Patient
* Patients can have anywhere between 1-6 rows in this file
* most patients have 2 rows (2682), followed closely by patients who have only 1 row (2309) and 3 rows (1563)
* patients with 4, 5, or 6 rows are very few in number (1, 1, and 2, respectively)

In [18]:
bish.groupby('PATID')['PATID'].count().unique()

array([2, 1, 3, 6, 4, 5])

In [77]:
bish.groupby('PATID')['PATID'].count().to_frame('Count').reset_index().groupby('Count').count()

,PATID
Count,
1,2309
2,2682
3,1563
4,1
5,1
6,2


## Rows per Patient in Udi's Subset (GA >= 238)
* In Udi's subset, patients have either 1 or 2 rows
* almost all the patients have 1 row (5367)
* 5 of the patients have 2 rows

In [21]:
bish.query('GA_In_Days > 238').groupby('PATID')['PATID'].count().unique()

array([1, 2])

In [82]:
bish.query('GA_In_Days > 238').groupby('PATID')['PATID'].count().to_frame('Count').reset_index().groupby('Count').count()

,PATID
Count,
1,5367
2,5


#### Which patients have 2 rows?

In [26]:
bish.query('GA_In_Days > 238').groupby('PATID')['PATID'].count().to_frame().query('PATID > 1')

,PATID
PATID,
S002-BISH-00496,2
S002-BISH-02007,2
S002-BISH-02008,2
S002-BISH-02826,2
S002-BISH-05059,2


#### When were these BISH exams taken (in GA days)?
* Only 1/5 of the patients have two distinct data 
* 4/5 patients have exams on the same data (are these just repeated records?)

In [91]:
doubles_index = bish.query('GA_In_Days > 238').groupby('PATID')['PATID'].count().to_frame().query('PATID > 1').index
bish.query('GA_In_Days > 238').set_index('PATID').loc[doubles_index,['GA_In_Days']].reset_index().groupby('PATID').agg([min,max])

GA_In_Days     
                       min  max
PATID                          
S002-BISH-00496        265  284
S002-BISH-02007        244  244
S002-BISH-02008        245  245
S002-BISH-02826        243  243
S002-BISH-05059        241  241

## Alternative Subset
What's weird about Udi's subset is that it happens at 34 weeks and above...  That is, at a time
when predicting the pregnancy becomes less and less critical.  

What if we instead restrict ourselves to anything before 34 weeks?
* We find that patients in this subset have anywhere between 1-4 records
* Of these patients, 2821 have only 1 record (SAD FACE)
* However, 1731 patients have 2 records
* 33 patients have 3 records, while 3 patients have 4
* So, 1767 patients have 2 records or more during this timeframe
* If we're ok with missing data, then we have 4588 records

In [113]:
bish.query('GA_In_Days < 238').groupby('PATID')['PATID'].count().unique()

array([2, 1, 3, 4])

In [114]:
bish.query('GA_In_Days < 238').groupby('PATID')['PATID'].count().to_frame('Count').reset_index().groupby('Count').count()

,PATID
Count,
1,2821
2,1731
3,33
4,3


### 2-Record Breakdown of Alternative Subset
Though there is a lot of nomenclature running around, like cycle IDs and Fx's, the patients 
with 2 exams do not necessarily follow...  

Udi left us w/ the following table of defs:

| Cycle ID | Description |
|----------|-------------|
| 5  | Enrollment        | 
| 10 | F1 - Week (20-24) |
| 15 | F2 - Week (28-32) |
| 20 | F3 - Week 34+     |

However, with two exams, we have the following:
* **Exam1** extends between **16.6 - 27.1 weeks**, though most of it occurs during weeks 20-24 
    - this is loosely the F1 exam in Udi's vernacular
* **Exam2** extends between **27 - 33.9 weeks**, though most of it occurs during weeks 28-32
    - this is loosely the F2 exam in Udi's vernacular
* No patient's 2 exams occur on the same day
    - minimum difference is 4 weeks
    - middle 50% is between 5.6 - 7.3 weeks
    - maximum difference is 14.5 weeks

In [152]:
doubles_index = bish.query('GA_In_Days < 238').groupby('PATID')['PATID'].count().to_frame().query('PATID == 2').index
two_records = bish.query('GA_In_Days < 238').set_index('PATID').loc[doubles_index,['GA_In_Days']].reset_index().groupby('PATID').agg([min,max]).GA_In_Days
two_records['max'] = round(two_records['max']/7, 1)
two_records['min'] = round(two_records['min']/7, 1)
two_records['diff'] = two_records['max'] - two_records['min']
two_records.head()

,min,max,diff
PATID,,,
S002-BISH-00001,21.4,28.9,7.5
S002-BISH-00009,20.3,28.4,8.1
S002-BISH-00015,21.9,28.9,7.0
S002-BISH-00017,20.3,29.0,8.7
S002-BISH-00020,20.0,31.0,11.0


In [153]:
len(two_records.query('diff == 0'))

0

In [155]:
# Patients w/ 2 records in interval 0<GA<34wks
two_records.describe().iloc[3:,:]

,min,max,diff
min,16.6,27.0,4.0
25%,22.1,28.4,5.6
50%,22.9,29.0,6.3
75%,23.3,29.9,7.3
max,27.1,33.9,14.5


### 3-Record Breakdown of Alternative Subset

With three exams, we have the following:
* **Exam1** extends between **18.6 - 28.3 weeks**, though most of it occurs during weeks 20-24 
    - this is loosely the F1-F2 exam range in Udi's vernacular (more closely matching F1)
* **Exam2** extends between **24.6 - 32.1 weeks**, though most of it occurs during weeks 28-32
    - this is loosely in the F2 exam range in Udi's vernacular
* **Exam3** extends between **31.3 - 33.9 weeks**
    - these are all F2 exams
* No patient's 2 exams occur on the same day
    - minimum difference between 2 consecutive exams is 1.6 weeks
    - maximum difference between 2 consecutive exams is 8.0 weeks
    - min (max) difference between 1st and 3rd exams is 5.4 wks (13.8 wks)

It's not clear why these patients have 3 exams like this, unless maybe the 
3rd one is approximately considered to be a cycle ID 20 (F3) exam.  


In [168]:
triples_index = bish.query('GA_In_Days < 238').groupby('PATID')['PATID'].count().to_frame().query('PATID == 3').index
three_records = bish.query('GA_In_Days < 238').set_index('PATID').loc[triples_index,['GA_In_Days']].reset_index().groupby('PATID').agg([min,np.median,max]).GA_In_Days
three_records['max'] = round(three_records['max']/7, 1)
three_records['median'] = round(three_records['median']/7, 1)
three_records['min'] = round(three_records['min']/7, 1)
three_records['med_max_diff'] = three_records['max'] - three_records['median']
three_records['min_med_diff'] = three_records['median'] - three_records['min']
three_records['min_max_diff'] = three_records['max'] - three_records['min']
three_records.head()

,min,median,max,med_max_diff,min_med_diff,min_max_diff
PATID,,,,,,
S002-BISH-00039,23.3,27.9,33.9,6.0,4.6,10.6
S002-BISH-00402,20.6,26.6,33.6,7.0,6.0,13.0
S002-BISH-00497,21.1,29.1,33.7,4.6,8.0,12.6
S002-BISH-00502,19.4,26.0,32.4,6.4,6.6,13.0
S002-BISH-00537,23.0,29.1,33.3,4.2,6.1,10.3


In [172]:
len(three_records.index)

33

In [169]:
len(three_records.query('med_max_diff == 0'))

0

In [170]:
len(three_records.query('min_med_diff == 0'))

0

In [171]:
# Patients w/ 3 records in interval 0<GA<34wks
three_records.describe().iloc[3:,:]

,min,median,max,med_max_diff,min_med_diff,min_max_diff
min,18.6,24.6,31.3,1.6,3.4,5.4
25%,20.9,27.6,33.4,5.2,4.9,10.6
50%,22.1,27.9,33.7,5.8,6.0,11.3
75%,23.0,28.3,33.9,6.0,6.7,12.8
max,28.3,32.1,33.9,7.8,8.0,13.8


In [173]:
set(three_records.index).intersection(two_records.index)

set()

#### How many Exam1 records occur during Cycle ID 10?

In [176]:
len(three_records.query('min > 19 & min < 25'))

31

#### How man Exam2 records occur during Cycle ID 10?

In [182]:
len(three_records.query('median > 19 & median < 25'))

1

#### How many Exam2 records occur during Cycle ID 15?

In [183]:
len(three_records.query('median > 27 & median < 33'))

29

#### How many Exam3 record occur during Cycle ID 15?

In [177]:
len(three_records.query('max > 27 & max < 33'))

3

In [ ]:
three_records.query('min > 19 & min < 25').index

# F3 Data: is too much being dropped?

If you look at the code, Udi is dropping any row of data that doesn't have
complete TOI data...  

```
# Load data
df = pickle.load( open( "data/interim/mbh001_Data-Wide-QC-full-data_20190924_V4_feateng.p", "rb" ) )

#Extracting f3 complete toi patids
f3_toi_patids = df.filter(regex="^toi.+20").dropna(axis=0).index # Dropping rows with missing data
```

But looking in the sensor files, you see most often people don't have 
complete data (e.g., they'll have everything but the `SC_X` vars).  Moreover, in the
final emails between Udi and David leading up to Udi's departure, David said to only trust
the log(bandPower) variables.

From David (Sept 26, 2019):
> If you calculate these proportions, with the current features, those are high quality: 
> * Low Frequency Band log Power/Total log Power = (toi.mvt_bandpwr2_log/toi.mvt_bandpwr1_log)
> * Mid Frequency Band log Power/Total log Power = (toi.mvt_bandpwr3_log/toi.mvt_bandpwr1_log) 
> * High Frequency Band log Power/Total log Power = (toi.mvt_bandpwr4_log/toi.mvt_bandpwr1_log)

Below, we find that
* Udi initially begins w/ 7,069 patient records
* he drops 4,262 of these records
* he ends the TOI-drop procedure w/ 2,807 records

We also find that Udi never even uses any of the features recommended by David as being
the only high quality TOI features.  That is, they are not even present in the "dropna" 
procedure he does... They never make it to Udi's repo/script from Carlos' repo/pipeline.  This
means we will have to look at the raw data exclusively (none of what Udi did here really
matters anymore).

In [100]:
# Get Udi's Starting Data Table
df = pickle.load( open( "../data/interim/mbh001_Data-Wide-QC-full-data_20190924_V4_feateng.p", "rb" ) )

In [101]:
toi_tbl = df.filter(regex="^toi.+20")
f3_toi_patids = toi_tbl.index
f3_toi_patids_drop = toi_tbl.dropna(axis=0).index # Dropping rows with missing data

In [110]:
print('Number of Patients:\n\t', len(f3_toi_patids))
print('\nNumber of Patients w/ at least 1 missing var:\n\t',len(f3_toi_patids_drop))
print('\nDifference:\n\t', len(f3_toi_patids) - len(f3_toi_patids_drop))


Number of Patients:
	 7069

Number of Patients w/ at least 1 missing var:
	 2807

Difference:
	 4262


In [190]:
num_toi = len(toi_tbl.columns)
num_toi

19

In [186]:
toi_tbl.columns

Index(['toi.f31f_hrmean1.20', 'toi.f31f_hrmed1.20', 'toi.f31f_hrstd1.20',
       'toi.f31f_hrsegs1.20', 'toi.f31f_totsegs1.20', 'toi.f32f_hrmean1.20',
       'toi.f32f_hrmed1.20', 'toi.f32f_hrstd1.20', 'toi.f32f_hrsegs1.20',
       'toi.f32f_totsegs1.20', 'toi.f3_medcc4.20', 'toi.f3_medlag4.20',
       'toi.f3_totsegs4.20', 'toi.f3_ccsegs4.20', 'toi.f3_mvt_pwr_mvt21.20',
       'toi.f3_mvt_pwr_mvt31.20', 'toi.f3_mvt_pwr_mvt41.20',
       'toi.f3_mvt_pwr_numbersegments.1.20',
       'toi.f3_mvt_pwr_numberusedpwrcoh.20'],
      dtype='object')

In [200]:
# TOI Missingness Per Record
toi_null = toi_tbl.isnull().sum(axis=1).to_frame('num_missing')
toi_null['percent_missing'] = round(toi_null['num_missing'] / num_toi, 2)
toi_null

,num_missing,percent_missing
patid,,
S002-BISH-00001,19,1.00
S002-BISH-00002,0,0.00
S002-BISH-00003,0,0.00
S002-BISH-00004,5,0.26
S002-BISH-00005,14,0.74
S002-BISH-00006,19,1.00
S002-BISH-00007,0,0.00
S002-BISH-00008,0,0.00
S002-BISH-00009,14,0.74


In [207]:
toi_tbl.loc[['S002-BISH-00004','S002-BISH-00014','S002-BISH-00015','S002-BISH-00017','S002-BISH-00018','S002-BISH-00021','S002-BISH-00022']]

,toi.f31f_hrmean1.20,toi.f31f_hrmed1.20,toi.f31f_hrstd1.20,toi.f31f_hrsegs1.20,toi.f31f_totsegs1.20,toi.f32f_hrmean1.20,toi.f32f_hrmed1.20,toi.f32f_hrstd1.20,toi.f32f_hrsegs1.20,toi.f32f_totsegs1.20,toi.f3_medcc4.20,toi.f3_medlag4.20,toi.f3_totsegs4.20,toi.f3_ccsegs4.20,toi.f3_mvt_pwr_mvt21.20,toi.f3_mvt_pwr_mvt31.20,toi.f3_mvt_pwr_mvt41.20,toi.f3_mvt_pwr_numbersegments.1.20,toi.f3_mvt_pwr_numberusedpwrcoh.20
patid,,,,,,,,,,,,,,,,,,,
S002-BISH-00004,NaN,NaN,NaN,NaN,NaN,140.18575,140.03575,4.814165,32.0,43.0,0.472417,-9.775,10.0,2.0,0.169885,0.137150,0.466450,10.0,1.0
S002-BISH-00014,NaN,NaN,NaN,NaN,NaN,132.26150,131.52650,4.440940,41.0,59.0,0.568553,-7.100,15.0,4.0,0.165809,0.100418,0.408182,15.0,4.0
S002-BISH-00015,NaN,NaN,NaN,NaN,NaN,134.47400,134.20400,4.671100,30.0,50.0,0.600217,-6.800,12.0,9.0,0.152383,0.152934,0.397743,12.0,9.0
S002-BISH-00017,NaN,NaN,NaN,NaN,NaN,159.65150,159.02450,4.593490,8.0,38.0,0.667281,-8.050,11.0,10.0,0.285043,0.164083,0.345901,11.0,8.0
S002-BISH-00018,NaN,NaN,NaN,NaN,NaN,131.52625,132.08525,5.602840,27.0,45.0,0.742802,-5.975,13.0,6.0,0.238292,0.112508,0.386279,13.0,7.0
S002-BISH-00021,NaN,NaN,NaN,NaN,NaN,144.42225,144.25375,5.854445,10.0,18.0,0.635892,-10.550,5.0,1.0,0.249220,0.096532,0.355166,5.0,2.0
S002-BISH-00022,NaN,NaN,NaN,NaN,NaN,130.56300,127.39000,8.078390,28.0,42.0,0.866797,-6.700,11.0,7.0,0.387365,0.088282,0.296879,11.0,8.0


### Analysis
1. Many times, a row has 5 missing TOI features.  By looking more closely, this often (if not always) 
   corresponds to the same 5 features: toi.f31f_hrmean1.20,	toi.f31f_hrmed1.20,	toi.f31f_hrstd1.20,
   toi.f31f_hrsegs1.20,	toi.f31f_totsegs1.20.  
   * Can drop these columns and use same dropna procedure, though this takes away a lot of
     good data
   * Can exclude these columns for the dropna procedure, then include them in the full data set
     (and treat the missing values somehow)
2. Note that 1F is for "quiet behavioral state", while 2F stands for  "active behavioral state"
   * Dani told me that a lot of 1F data is actually missing
   * When I asked her (in Slack), "Should we combine 1F and 2F data to reduce sparsity?", 
     Dani responded: "I think they are very  very different. I would use just the 2f, I think that is
     what has more data."
   * So basically, dropping these columns would be ok with her.
2. Note that Udi never touches the sensor files in his repo.  They are ported from Carlos' 
   repo via the "Full/Wide" data table the Udi imports into his first script.  Due to
   whatever pipeline the sensor features are run through, the band power features that
   David recommended as the only high quality TOI features do not even make it into
   Udi's script here...  

# Recap of the TOI Data

MON data gets the shortest end of the stick, technically, since only the MON data
that corresponds with good TOI data is looked at -- and from there, only the good MON data
is kept.

Below, here is a blow-by-blow of how it all happens.

### Load Full Data Set
If you remember, two files are loaded: the Data-Wide-QC-Full table and the BISH-FETAL-GA table. 

For the BISH table, Udi first looks only at those TOI.20 features that made it through Carols'
pipeline.  This is the first unfortunate mistake, as none of the features recommended by David
as being high quality make it this far.

Udi then does a complete case analysis on the TOI subtable, dropping any row that does not have all
TOI.20 features.  This is the next unfortunate mistake, as many of the dropped rows here were 
simply missing "1F" data.  Dani found out that "1F" and "2F" stand for "quiet" and "active" fetal
states -- and that most people simply do not have "1F" data (baby was active during TOI 
measurements).  This represent a huge and unnecessary amount data discarded.


```
# Load Data
df = pickle.load( open( "data/interim/mbh001_Data-Wide-QC-full-data_20190924_V4_feateng.p", "rb" ) )

# Extract F3 complete TOI patids
f3_toi_patids = df.filter(regex="^toi.+20").dropna(axis=0).index # Dropping rows with missing data
df_f3_toi = df.loc[f3_toi_patids,:]
```

Later in the code, Udi also drops any pregnancy w/ twins.

```
# Drop twins
df_f3_toi=df_f3_toi[df_f3_toi['NumberOfFetuses']==1]   # Keep only single fetus
df_f3_toi.drop('NumberOfFetuses',axis=1,inplace=True)  # No longer need the var

# Rename a var for convenience
df_f3_toi.rename(columns = {'adsc.dlvryga_dys.25':'ga.birth'}, inplace=True)
```


### Load BISH session records
The next thing Udi does is load the BISH data, which includes the estimated GA at
the visitation level.  This means that, in the full file, PATIDs are listed in as many
rows as the patient had visits.  However, Udi then filters the table to look only
at rows where the estimated GA is greater than 238 days (equal to 34 weeks).  Perhaps
accidentally, perhaps intentionally, this helps ensure that (for the most part) patients
have only one record in the subtable.  The point of the subset though was because he
was interested in predicting the "time left" from the doctor's visit during F3 to the 
time of birth.

```
# GA at F3 for each patient
collect_times = pd.read_csv("data/raw/BISH FETAL GA ASSESSMENT.csv")
collect_times.columns = collect_times.columns.str.replace(' ', '')
collect_times.rename(columns = {'GA_In_Days':'ga_in_days',"PATID":'patid'},inplace=True)
collect_times_f3 = collect_times[collect_times.ga_in_days>=238].loc[:,["ga_in_days","patid","NumberOfFetuses"]]
collect_times_f3.rename(columns = {'ga_in_days':'ga_f3'},inplace=True)
```



### Combine BISH F3 Subset w/ Full Data Set

```
# Subset of features with relevant cycle id 
df_f3_toi=df_f3_toi.merge(collect_times_f3,on='patid')
```

Right after this is when he drops the twins (code shown in above section).


He then computes the target variable he is interested in and ensure no funky/erroneous
records are left in the set:
```
# Create target variable (time left)
df_f3_toi['time_left'] = df_f3_toi['ga.birth']-df_f3_toi['ga_f3']
df_f3_toi['time_left'] = df_f3_toi['time_left'].astype('float64')
df_f3_toi = df_f3_toi[df_f3_toi['time_left'] >= 0]

# Here, he just ensures that ga_f3 is of type float64 
df_f3_toi['ga_f3'] = df_f3_toi['ga_f3'].astype('float64')

# Here, he just ensures that the set is indexed by patid
df_f3_toi.set_index(['patid'], inplace=True)
```

Finally, he replaces any missing values marked as -99999 or -999999 with `None`.

```
# Replacing artifacts with missing values
artifact_val1=-99999
artifact_val2=-999999
df_f3_toi.replace(artifact_val1,None,inplace=True)
df_f3_toi.replace(artifact_val2,None,inplace=True)
```


### Save TOI File
Initially, Udi was saving a unique data file every time he ran his script.  This was
done by appending the current datetime to the file name.  However, he ran the script
a lot and realized he didn't need to keep saving the file each time he fixed a bug (you
can see all the files in his `data/processed` folder).  So, at one point, he 
commented out the datetime and left the variable with the value `a` -- this means that
the most recent version of his processed data is the file appended with `a`.

```
# Get current timestamp
now = datetime.now()
date_time = 'a'#now.strftime("%Y%m%d%H%M%S")
toi_fname = 'data/processed/mbh001_f3_toi_baseline_{}'.format(date_time)

# Create TOI CSV File
print('Writing file:{}'.format(toi_fname+'.csv'))
df_f3_toi.to_csv(toi_fname+'.csv')

# Create TOI Pickle File
print ('Writing file:{}'.format(toi_fname+'.p'))
pickle.dump(df_f3_toi, open(toi_fname+'.p', "wb" ) )
```


In [238]:
def udi_toi_table():
    # Load Data
    df = pickle.load( open( "../data/interim/mbh001_Data-Wide-QC-full-data_20190924_V4_feateng.p", "rb" ) )

    # Extract F3 complete TOI patids
    f3_toi_patids = df.filter(regex="^toi.+20").dropna(axis=0).index # Dropping rows with missing data
    df_f3_toi = df.loc[f3_toi_patids,:]


    # Rename a var for convenience
    df_f3_toi.rename(columns = {'adsc.dlvryga_dys.25':'ga.birth'}, inplace=True)

    # GA at F3 for each patient
    collect_times = pd.read_csv("../data/raw/BISH FETAL GA ASSESSMENT.csv")
    collect_times.columns = collect_times.columns.str.replace(' ', '')
    collect_times.rename(columns = {'GA_In_Days':'ga_in_days',"PATID":'patid'},inplace=True)
    collect_times_f3 = collect_times[collect_times.ga_in_days>=238].loc[:,["ga_in_days","patid","NumberOfFetuses"]]
    collect_times_f3.rename(columns = {'ga_in_days':'ga_f3'},inplace=True)
    
    # Subset of features with relevant cycle id 
    df_f3_toi=df_f3_toi.merge(collect_times_f3,on='patid')
    
    # Drop twins
    df_f3_toi = df_f3_toi[df_f3_toi['NumberOfFetuses']==1]   # Keep only single fetus
    df_f3_toi.drop('NumberOfFetuses',axis=1,inplace=True)  # No longer need the var

    # Create target variable (time left)
    df_f3_toi['time_left'] = df_f3_toi['ga.birth']-df_f3_toi['ga_f3']
    df_f3_toi['time_left'] = df_f3_toi['time_left'].astype('float64')
    df_f3_toi = df_f3_toi[df_f3_toi['time_left'] >= 0]

    # Here, he just ensures that ga_f3 is of type float64 
    df_f3_toi['ga_f3'] = df_f3_toi['ga_f3'].astype('float64')

    # Here, he just ensures that the set is indexed by patid
    df_f3_toi.set_index(['patid'], inplace=True)

    # Replacing artifacts with missing values
    artifact_val1=-99999
    artifact_val2=-999999
    df_f3_toi.replace(artifact_val1,None,inplace=True)
    df_f3_toi.replace(artifact_val2,None,inplace=True)
    
    return df_f3_toi

In [276]:
df_f3_toi = udi_toi_table()

In [278]:
nrows_df, ncols_df = df.shape
nrows_toi, ncols_toi = df_f3_toi.shape

print('After Pipeline (rows, cols):', nrows_df, ncols_df)
print('After Udi (Toi Table):', nrows_toi, ncols_toi)

After Pipeline (rows, cols): 7069 575
After Udi (Toi Table): 2732 577


In [279]:
set(df_f3_toi.columns).difference(df.columns)

{'ga.birth', 'ga_f3', 'time_left'}

In [280]:
set(df.columns).difference(df_f3_toi.columns)

{'adsc.dlvryga_dys.25'}

Remember, `adsc.dlvryga_dys.25` was renamed to `ga.birth`, so technically the only
difference in columns between these two tables is that Udi added `ga_f3` and `time_left`.

# MON Data
After Udi saves the TOI data, he then makes one last table, which is a TOI/MON intersection 
table.

To do this, he takes the TOI table he just created and searches it for all MON.20 vars.

```
# TOI_MON intersection
df_mon = df_f3_toi.filter(regex="^mon.+20")
```

Udi then creates two more Null Filters:
* the MON Feature Null Filter
* the MON Record Null Filter

Note that for TOI, Udi just has a record-wise null filter, where any records with at least
one missing TOI feature was dropped.  MON features must be really sparse: this explains why
he first gets rid of the most sparse features before doing a record-wise drop.  Still: there
is no treatment of missing values, after the feature-wise filter, he applies a CCA-style
record-wise filter.  (Btw, after all is said and done with TOI and MON, Udi applies a final "Feature Null Filter", 
which is a column-wise null filter that he applies in his Jupyter Notebook right before
model building.)


For TOI, he dropped any record that did not have COMPLETE data.  He is a little more
lenienent on MON, where he allows a MON feature to be up to 55\% null.

```
# MON Feature-Wise Null Filter

## Order MON features by how Null they are
df_mon_na_frac = (df_mon.isnull().sum()/len(df_mon)).sort_values()

## Set a Null threshold
mon_na_th=0.55

## Only keep good MON features (e.g., nullity <= 55%)
mon_features = df_mon_na_frac[df_mon_na_frac <= mon_na_th].index

# MON Record-Wise Null Filter
f3_toi_mon_patids = df_f3_toi[mon_features].dropna(axis=0).index # Dropping rows with missing data
df_f3_toi_mon = df_f3_toi.loc[f3_toi_mon_patids,:]
```

Udi then saves a CSV and pickle file for this TOI/MON table.


```
toi_mon_fname='data/processed/mbh001_f3_toi_mon_baseline_{}'.format(date_time)

# Save a CSV File
print ('Writing file:{}'.format(toi_mon_fname+'.csv'))
df_f3_toi_mon.to_csv(toi_mon_fname+'.csv')

# Save a Pickle File
print ('Writing file:{}'.format(toi_mon_fname+'.p'))
pickle.dump(df_f3_toi_mon, open(toi_mon_fname+'.p', "wb" ) )
```



In [281]:
# TOI_MON intersection
df_mon = df_f3_toi.filter(regex="^mon.+20")

In [282]:
# MON Feature-Wise Null Filter
## Order MON features by how Null they are
df_mon_na_frac = (df_mon.isnull().sum()/len(df_mon)).sort_values()

## Set a Null threshold
mon_na_th=0.55

## Only keep good MON features (e.g., nullity <= 55%)
mon_features = df_mon_na_frac[df_mon_na_frac <= mon_na_th].index

# MON Record-Wise Null Filter
f3_toi_mon_patids = df_f3_toi[mon_features].dropna(axis=0).index # Dropping rows with missing data
df_f3_toi_mon = df_f3_toi.loc[f3_toi_mon_patids,:]

In [284]:
df_f3_toi_mon.shape

(1548, 577)

# Misc: Ad Hoc Look-Ups for Dani

Dani found some odd non-BISH IDs in one of the data tables.  We are only
supposed to used BISH IDs. 

**Question**:  Did Udi use any non-BISH IDs?

**Answer**: No.

In [209]:
[item for item in df.index if 'BISH' not in item]

[]

**Question**: Dani is interested in the PTSD data.  Wants to know:  Did Udi
use any of the PTSD data?

**Answer**:  PTSD vars were certainly in the pipeline, however, the PTSD vars do NOT make it past the null filter.

In [210]:
[item for item in df.columns if 'pts' in item.lower()]

['adptsd.htq_dsm_iv_raw.15',
 'adptsd.htq_dsm_iv_raw.20',
 'adptsd.htq_functioning_raw.15',
 'adptsd.htq_functioning_raw.20',
 'adptsd.htq_total_raw.15',
 'adptsd.htq_total_raw.20',
 'adptsd.lec_events.15',
 'adptsd.lec_events.20',
 'adptsd.lec_exp.15',
 'adptsd.lec_exp.20',
 'adptsd.lec_exp_nmiss.15',
 'adptsd.lec_exp_nmiss.20',
 'adptsd.ptsdsymptomatic2.15',
 'adptsd.ptsdsymptomatic2.20']

In [230]:
[item for item in df.dropna(axis=1, thresh=0.8*len(df)).columns 
     if 'pts' in item.lower()]

[]

**Question**:  Dani wanted to know if Udi included any data from the ADSC table.

**Answer**:  ADSC data was inded in the pipeline.  Also: many of the vars made it past Udi's
null filter.  

In [211]:
[item for item in df.columns if 'adsc' in item.lower()]

['adsc.dlvryga_dys.25',
 'adsc.gender2.25',
 'adsc.infraceai4.25',
 'adsc.infraceai7.25',
 'adsc.infracenih4.25',
 'adsc.infracenih7.25',
 'adscmat.educ2.any',
 'adscmat.educ4.any',
 'adscmat.educ6.any',
 'adscmat.educ7.any',
 'adscmat.educ_combohs4.any',
 'adscmat.educ_ptnr2.any',
 'adscmat.educ_ptnr4.any',
 'adscmat.educ_ptnr6.any',
 'adscmat.educ_ptnr7.any',
 'adscmat.empl.any',
 'adscmat.empl2.any',
 'adscmat.empl_comb4.any',
 'adscmat.empl_ptnr2.any',
 'adscmat.fertility2.any',
 'adscmat.fertilityart2.any',
 'adscmat.fertilityoi2.any',
 'adscmat.govsupport4.any',
 'adscmat.govsupportpre2.any',
 'adscmat.govsupportpreall2.any',
 'adscmat.gravidity.any',
 'adscmat.grossincome7.any',
 'adscmat.healthcvg4.any',
 'adscmat.inc_pp.any',
 'adscmat.marriedlt2.any',
 'adscmat.marrpart2.any',
 'adscmat.nulliparous2.any',
 'adscmat.parity.any',
 'adscmat.precareall_2.any',
 'adscmat.precarefirstga.any',
 'adscmat.precaret1_2.any',
 'adscmat.publicassist2.any',
 'adscmat.publicassistfin2.any',

In [231]:
[item for item in df.dropna(axis=1, thresh=0.8*len(df)).columns 
     if 'adsc' in item.lower()]

['adsc.dlvryga_dys.25',
 'adsc.gender2.25',
 'adsc.infraceai4.25',
 'adsc.infraceai7.25',
 'adsc.infracenih4.25',
 'adsc.infracenih7.25',
 'adscmat.educ2.any',
 'adscmat.educ4.any',
 'adscmat.educ6.any',
 'adscmat.educ7.any',
 'adscmat.empl2.any',
 'adscmat.empl_ptnr2.any',
 'adscmat.fertility2.any',
 'adscmat.fertilityart2.any',
 'adscmat.fertilityoi2.any',
 'adscmat.govsupport4.any',
 'adscmat.govsupportpreall2.any',
 'adscmat.gravidity.any',
 'adscmat.healthcvg4.any',
 'adscmat.marriedlt2.any',
 'adscmat.marrpart2.any',
 'adscmat.nulliparous2.any',
 'adscmat.parity.any',
 'adscmat.precareall_2.any',
 'adscmat.precarefirstga.any',
 'adscmat.precaret1_2.any',
 'adscmat.publicassist2.any',
 'adscmat.publicassistfin2.any',
 'adscmat.selfpay2.any',
 'adscmat.toiletwater2.any']